In [1]:
import os
import torch
import numpy as np
from termcolor import colored
import pickle as pkl
import copy

#from my_model import MyModel
from utils import kw_rake, Scoring, end_process, evaluation, get_input, get_output
from dataset import para_data_bidirect
from model import MyModel

from tqdm import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration


In [2]:
#max_length = 75
top_k = 50
top_p = 0.92
n_beams = 1
early_stopping = False
do_sample = True
temperature = 1.5

cand_number = 100
test_number = 5
score_k = 5

In [3]:
#model_name = 'bart-base'
model_name = 't5-large'
if_bart = model_name[:4]=='bart'

#my_pretrained_root = '/data/ljx/cpt/KQAPro_ckpt/program_ckpt/'
my_pretrained_root = '/data/MODELS/t5-large'
my_model = MyModel(model_name, pretrained_dir = my_pretrained_root)

data_path = '/data/ljx/data/bookcorpus/para/'
Data = para_data_bidirect(data_path)

print(colored("Output (top-{}):".format(score_k), 'red'))
# Generate
print(colored("top_k       = {}".format(top_k), 'yellow'))
print(colored("top_p       = {}".format(top_p), 'yellow'))
print(colored("n_beams     = {}".format(n_beams), 'yellow'))
print(colored("do_sample   = {}".format(do_sample), 'yellow'))
print(colored("temperature = {}".format(temperature), 'yellow'))

actual_s = []
generated_s = []
data_use = Data.data[:test_number]

Loading model from /data/MODELS/t5-large
Loading cache from /data/ljx/data/bookcorpus/para/bidirect_context_window3_sample10000.pkl
Output (top-5):
top_k       = 50
top_p       = 0.92
n_beams     = 1
do_sample   = True
temperature = 1.5


In [4]:
for sample in tqdm(data_use):
    context_len = len(sample)
    mid_id = round((context_len - 1)/2)
    
    X = sample[mid_id]
    Y_ref = sample[mid_id-1:mid_id+1]
    actual_s.append(X)
    
    print(colored("Input: "+X, 'yellow'))

    kw_batches = kw_rake(X)
    X_cands = []
    Y_cands = []
    for i in range(cand_number): 
        inputs, X_processed = get_input(sample, mid_id, kw_batches)
        input_ids = my_model.encode(inputs).input_ids
        
        output = my_model.model.generate(
            input_ids=input_ids, 
            do_sample=do_sample,
            num_beams=n_beams,
            top_k=top_k,
            top_p=top_p,
            early_stopping = early_stopping,
            temperature=temperature
           )
        
        output = my_model.tokenizer.batch_decode(output)
        output = output[0]
        Y, X_decoded = get_output(sample, X_processed, output)
        
        X_cands.append(X_decoded)
        Y_cands.append(Y)

    text_top_k = Scoring(X, X_cands, Y, Y_cands, score_k)
    generated_s.append(text_top_k[0])
    for i, l in enumerate(text_top_k):
        print(('%s. %s' % (str(i+1), l)))

  0%|          | 0/5 [00:00<?, ?it/s]

Input: maybe i could stay home and sleep one night .


  0%|          | 0/5 [00:57<?, ?it/s]


NameError: name 'cands' is not defined